Importing modules from conda environment '/lustre/scratch118/malaria/team222/jr35/phd/envs_py/scv_jupcon/'

In [ ]:
import seaborn as sns
import pandas as pd
## import scrublet as scr
import leidenalg
import matplotlib.pyplot as plt
import numpy as np
import scvelo as scv
import scanpy as sc
import scipy.io
import loompy as lp
import anndata
import cellrank as cr
from cellrank.tl.kernels import VelocityKernel
from cellrank.tl.kernels import PseudotimeKernel
from cellrank.tl.kernels import ConnectivityKernel
from cellrank.tl.estimators import GPCCA
import bbknn
import copy

In [ ]:
import plotly.express as px
import pickle
from IPython.display import display
from upsetplot import generate_counts, plot 
import string


In [ ]:
import warnings
import re
# warnings.filterwarnings(action='ignore')
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)

In [ ]:
import os
os.chdir('/Users/jr35/Google Drive/My Drive/PHD_documents/malaria_phd/sc_velo/')

In [ ]:
os.getcwd()

In [ ]:
# import warnings

# def function_that_warns():
#     warnings.warn("deprecated", DeprecationWarning)

# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
#     function_that_warns()  # this will not show a warning


Check module version

In [ ]:
# print(cr.__version__)

In [ ]:
# unload module
# del str

In [ ]:
scv.logging.print_versions()

set some figure settings

In [ ]:
scv.set_figure_params(transparent=False, facecolor = 'white', frameon=True)

Initialize variables for some where the raw and processed data is

In [ ]:
raw_data = 'data/raw/PfDB52e/'
processed_data = 'data/processed/PfDB52e/'

In [ ]:
raw_data

In [ ]:
##Read in in splice and unspliced matrices genereted by velocyto

In [ ]:
# from pathlib import Path
# result = list(Path("data/processed/PfDB52e").rglob("*.[lL][oO][oO][mM]"))

In [ ]:
# result

Combine day10 looms

In [ ]:
# loompy.combine(['data/processed/PfDB52e/day10/vcyto/possorted_genome_bam_7YCS2.loom', 'data/processed/PfDB52e/day10b/vcyto/possorted_genome_bam_09PD6.loom'], '/lustre/scratch118/malaria/team222/jr35/phd/V3_4TP/sc_velodata/processed/PfDB52e/d10_d10b/vcyto/possorted_genome_bam_d10_d10b.loom', key="Accession")

Combine all looms

In [ ]:
# loompy.combine(['data/processed/PfDB52e/asexuals/vcyto/possorted_genome_bam_UZNXI.loom', 'data/processed/PfDB52e/day3/vcyto/possorted_genome_bam_ASHFZ.loom', 'data/processed/PfDB52e/day5/vcyto/possorted_genome_bam_H66NQ.loom','data/processed/PfDB52e/day10/vcyto/possorted_genome_bam_7YCS2.loom', 'data/processed/PfDB52e/day10b/vcyto/possorted_genome_bam_09PD6.loom'], '/lustre/scratch118/malaria/team222/jr35/phd/V3_4TP/sc_velodata/processed/PfDB52e/all_days/vcyto/possorted_genome_bam_all_days.loom', key="Accession")

In [ ]:
# d10_d10b_loom = scv.read_loom('/lustre/scratch118/malaria/team222/jr35/phd/V3_4TP/sc_velodata/processed/PfDB52e/d10_d10b/vcyto/possorted_genome_bam_d10_d10b.loom')

In [ ]:
# d5_filt_lp = scv.read_loom(processed_data + "vcyto/possorted_genome_bam_H66NQ.loom")

Read both day10s, format barcode names and combine and save as loom

Initialize variables for the different datasets/batches (days when sample was processed)

In [ ]:
# v3_days_d10 = ['day10', 'day10b']

Initialize variables for the loom files with spliced and unspliced matrices generated by velocyto for each of the different datasets/batches (days when sample was processed)

In [ ]:
# lp_name_d10 = [ 'possorted_genome_bam_7YCS2.loom', 'possorted_genome_bam_09PD6.loom']

Reading in the loom files with spliced and unspliced matrices and formating barcodes to allow merging with seurat object

In [ ]:
# bcode_days_d10 = ['D10a','D10b']

In [ ]:
# filt_lp_d10 = []
# for day, lp, v3_day in zip(v3_days_d10, lp_name_d10, bcode_days_d10):
#     x = scv.read_loom('data/processed/PfDB52e/' + day + '/vcyto/' + lp)
#     # scv.utils.clean_obs_names(x, alphabet='[AGTC]', id_length=12, inplace=True)
#     x.obs_names = [v3_day+'_'+re.sub('^.*:|x$', '', i)+'-1' for i in x.obs_names]
#     x.write_loom('data/processed/PfDB52e/' + day + '/vcyto/' + 'barcode_fmt_'+lp)
#     filt_lp_d10.append(x)

In [ ]:
# loompy.combine(['data/processed/PfDB52e/day10/vcyto/barcode_fmt_possorted_genome_bam_7YCS2.loom', 'data/processed/PfDB52e/day10b/vcyto/barcode_fmt_possorted_genome_bam_09PD6.loom'], '/lustre/scratch118/malaria/team222/jr35/phd/V3_4TP/sc_velodata/processed/PfDB52e/d10_d10b/vcyto/barcode_fmt_possorted_genome_bam_d10_d10b.loom', key="Accession")

In [ ]:
##Copy loom files from farm to local using rsync (Had to set up ssh keys to enable the copying without asking for password)

In [ ]:
# %%bash
# destination_day=("asexuals" "day3" "day5" "day10" "day10b" "d10_d10b" "all_days")
# lp_name=('possorted_genome_bam_UZNXI.loom' 'possorted_genome_bam_ASHFZ.loom' 'possorted_genome_bam_H66NQ.loom' 'possorted_genome_bam_7YCS2.loom' 'possorted_genome_bam_09PD6.loom' 'possorted_genome_bam_d10_d10b.loom' 'possorted_genome_bam_all_days.loom')
# # w_dir='/Volumes/GoogleDrive/My\ Drive/PHD_documents/malaria_phd/sc_velodata'
# # echo $w_dir
# for j in "${!destination_day[@]}";
#     do
#         mkdir -p /Volumes/GoogleDrive/My\ Drive/PHD_documents/malaria_phd/sc_velodata/processed/PfDB52e/${destination_day[j]}/vcyto/
#         rsync -av jr35@farm5-head1:/lustre/scratch118/malaria/team222/jr35/phd/V3_4TP/sc_velodata/processed/PfDB52e/${destination_day[j]}/vcyto/${lp_name[j]} /Volumes/GoogleDrive/My\ Drive/PHD_documents/malaria_phd/sc_velodata/processed/PfDB52e/${destination_day[j]}/vcyto/
#     done
    

Initialize variables for the different datasets/batches (days when sample was processed)

In [ ]:
v3_days = ['asexuals', 'day3', 'day5', 'day10', 'day10b', 'd10_d10b', 'all_days']

In [ ]:
v3_days_all = ['asexuals', 'day3', 'day5', 'day10', 'day10b', 'd10_d10b', 'all_days', 'all']

Initialize variables for the loom files with spliced and unspliced matrices generated by velocyto for each of the different datasets/batches (days when sample was processed)

In [ ]:
lp_name = ['possorted_genome_bam_UZNXI.loom', 'possorted_genome_bam_ASHFZ.loom', 'possorted_genome_bam_H66NQ.loom', 'possorted_genome_bam_7YCS2.loom', 'possorted_genome_bam_09PD6.loom', 'possorted_genome_bam_d10_d10b.loom', 'possorted_genome_bam_all_days.loom']

Reading in the loom files with spliced and unspliced matrices and formating barcodes to allow merging with seurat object

In [ ]:
bcode_days = ['D0','D3','D5','D10a','D10b']

In [ ]:
# filt_lp=[]

# for j, lp in enumerate(lp_name):
#     if j in range(0,5):
#         x = scv.read_loom('data/processed/PfDB52e/' + v3_days[j] + '/vcyto/' + lp)
#         # scv.utils.clean_obs_names(x, alphabet='[AGTC]', id_length=12, inplace=True)
#         x.obs_names = [bcode_days[j]+'_'+re.sub('^.*:|x$', '', i)+'-1' for i in x.obs_names]
#         filt_lp.append(x)
#     else :
#         x = scv.read_loom('data/processed/PfDB52e/' + v3_days[j] + '/vcyto/' + lp)
#         # scv.utils.clean_obs_names(x, alphabet='[AGTC]', id_length=12, inplace=True)
#         x.obs_names = [i.replace('possorted_genome_bam_UZNXI:', 'D0_').replace('possorted_genome_bam_ASHFZ:', 'D3_').replace('possorted_genome_bam_H66NQ:', 'D5_').replace('possorted_genome_bam_7YCS2:', 'D10a_').replace('possorted_genome_bam_09PD6:', 'D10b_').replace('x','-1')  for i in x.obs_names] 
#         filt_lp.append(x)

In [ ]:
filt_lp = scv.read_loom('data/processed/PfDB52e/all_days/vcyto/possorted_genome_bam_all_days.loom')

In [ ]:
filt_lp.obs_names = [i.replace('possorted_genome_bam_UZNXI:', 'D0_').replace('possorted_genome_bam_ASHFZ:', 'D3_').replace('possorted_genome_bam_H66NQ:', 'D5_').replace('possorted_genome_bam_7YCS2:', 'D10a_').replace('possorted_genome_bam_09PD6:', 'D10b_').replace('x','-1')  for i in filt_lp.obs_names] 

In [ ]:
filt_lp.obs_names

checking for duplicates in cell barcodes for merged day10 loom files as it is common for different batches to share the same barcodes

In [ ]:
from collections import Counter
cc = Counter(filt_lp.obs_names)
cc.most_common(10)

In [ ]:
##scVelo using seurat umap

Reading the raw data QCd by Elias

In [ ]:
ef_qcd = sc.read_h5ad(processed_data + '../RoadtoUMAP/Road2UMAP_raw.h5ad')

In [ ]:
ef_qcd_raw = ef_qcd.raw.to_adata().copy()

In [ ]:
# print(ef_qcd_raw.X)

In [ ]:
ef_qcd_raw.obs.value_counts('Stage')

In [ ]:
type(ef_qcd_raw.obs['Stage'])

Cleaning cell barcodes to remove the strings added at the beginning and end to allow merging with loom files containing spliced and unspliced counts

In [ ]:
def subset_seu_data(stages, day):
    subset_ad = ef_qcd_raw[(ef_qcd_raw.obs['Stage'].isin(stages)) & (ef_qcd_raw.obs['orig.ident'] == day), ]
    # scv.utils.clean_obs_names(subset_ad)
    # scv.utils.clean_obs_names(subset_ad, alphabet='[AGTC]', id_length=12)
    return subset_ad

In [ ]:
v3_days

In [ ]:
ef_qcd_raw.obs['stage_bfs'] = ef_qcd_raw.obs['anno_300cells']
ef_qcd_raw.obs = ef_qcd_raw.obs.astype({"stage_bfs": str}, errors='raise')
ef_qcd_raw.obs.loc[ef_qcd_raw.obs['stage_bfs'] == 'no assignment', 'stage_bfs'] = ef_qcd_raw.obs['Stage']
ef_qcd_raw.obs.loc[ef_qcd_raw.obs['stage_bfs'] == 'Gametocytes (comitted)', 'stage_bfs'] = 'Gametocytes (committed)'
ef_qcd_raw.obs['stage_bfs'] = ef_qcd_raw.obs['stage_bfs'].astype('category')

In [ ]:
asx_gams_bf1_pts = pd.read_csv('data/processed/RoadtoUMAP/pseudotime_tradeseq/asx_gams_bf1_pts.csv')
gams_bf2_pts = pd.read_csv('data/processed/RoadtoUMAP/pseudotime_tradeseq/gams_bf2_pts.csv')

In [ ]:
gams_bf2_pts.plot.scatter('slingPseudotime_1', 'slingPseudotime_2')

In [ ]:
asx_gams_bf1_pts.plot.scatter('slingPseudotime_1', 'slingPseudotime_2')

In [ ]:
ef_bf1_bf2=[]
for i in range(2):
    if i == 0:
        x=ef_qcd_raw[ef_qcd_raw.obs['stage_bfs'].isin(['late ring', 'early trophozoite', 'Gametocytes (committed)', 'Gametocytes (developing)','late trophozoite'])& ~ef_qcd_raw.obs['Stage'].isin(['late schizont', 'early ring'])]
        x.obs = pd.merge(x.obs,  asx_gams_bf1_pts.set_index('bcode'), how= 'left', left_index=True, right_index=True)
        x.obs['pseudo_mg'] = x.obs.slingPseudotime_2.fillna(x.obs.slingPseudotime_1)
        ef_bf1_bf2.append(x)
    else:
        x=ef_qcd_raw[ef_qcd_raw.obs['stage_bfs'].isin(['Gametocytes (developing)', 'Gametocytes (branching)', 'Gametocytes (early female)', 'Gametocytes (late female)', 'Gametocytes (early male)', 'Gametocytes (late male)'])]
        x.obs = pd.merge(x.obs,  gams_bf2_pts.set_index('bcode'), how= 'left',  left_index=True, right_index=True)
        x.obs['pseudo_mg'] = x.obs.slingPseudotime_2.fillna(x.obs.slingPseudotime_1)
        ef_bf1_bf2.append(x)
   

In [ ]:
# ef_qcd_raw[ef_qcd_raw.obs['stage_bfs'].isin(['late ring', 'early trophozoite', 'Gametocytes (committed)', 'Gametocytes (developing)','late trophozoite'])].obs[['Stage','stage_bfs']]

In [ ]:
# ef_qcd_raw[ef_qcd_raw.obs['stage_bfs'].isin(['late ring', 'early trophozoite', 'Gametocytes (committed)', 'Gametocytes (developing)','late trophozoite']) & ~ef_qcd_raw.obs['Stage'].isin(['late schizont', 'early ring'])].obs[['Stage','stage_bfs']]

In [ ]:
# ef_bf1_bf2[0][ef_bf1_bf2[0].obs.pseudo_mg.isna()].obs[['Stage','stage_bfs']]

Plotting umap of the subsetted datasets

In [ ]:
for inp in ef_bf1_bf2:
    inp.obs['Stage'] = inp.obs['Stage'].astype('category')
    inp.obs['orig.ident'] = inp.obs['orig.ident'].astype('category')
    scv.pl.umap(inp, color='Stage', components='2,3')
    # print(type(inp.obs['Stage']))

Plotting 3D umap for one example

In [ ]:
umap = ef_bf1_bf2[1].obsm['X_umap']

In [ ]:
fig_3d = px.scatter_3d(x =umap[:, 0], y = umap[:, 1], z = umap[:, 2], color = ef_bf1_bf2[1].obs.stage_bfs)

In [ ]:
fig_3d.update_traces(marker_size=1)


fig_3d.show()

QC checks - Tabulating the number of Stage categories, check for duplicates and glimpse the counts

In [ ]:
ef_bf1_bf2[1].obs['Stage'].value_counts()

In [ ]:
for obj1 in ef_bf1_bf2:
    print(np.count_nonzero(obj1.obs_names.duplicated()))

In [ ]:
print(np.count_nonzero(filt_lp.obs_names.duplicated()))

In [ ]:
for obj1 in ef_bf1_bf2:
    print(np.count_nonzero(obj1.obs_names.duplicated()))
    print(len(obj1.var_names))
    print(obj1.var_names[1:2])

In [ ]:
print(np.count_nonzero(filt_lp.obs_names.duplicated()))
print(len(filt_lp.var_names))
print(filt_lp.var_names[1:2])

In [ ]:
print(filt_lp.X[1:5,1:20])
    

Replacing '-' in variable/gene names with '_'

In [ ]:
for obj1 in ef_bf1_bf2:
    obj1.var_names = [i.replace("-", "_") for i in obj1.var_names]
#     obj.var_names = [i.replace("-", "_") for i in obj.var_names]
# ef_qcd_bf1_d5.var_names = [i.replace("-", "_") for i in ef_qcd_bf1_d5.var_names]

In [ ]:
for obj1 in ef_bf1_bf2:
    print(obj1.var_names[1:2])

In [ ]:
ef_bf1_bf2

In [ ]:
filt_lp.obs_names

In [ ]:
filt_lp

Merge subsetted QCd objects from Elias with loom files with unspliced/spliced matrices

The columns "initial_size", "initial_size_unspliced" and "initial_size_spliced" are added when calling scvelo.utils.merge. These are the counts per cell prior to subsetting, i.e. the initial size of the cell.

In [ ]:
ef_bf1n2_s_us = []
for obj in ef_bf1_bf2:
    x = scv.utils.merge(obj, filt_lp)
    ef_bf1n2_s_us.append(x)

In [ ]:
ef_bf1n2_s_us

QC checks

In [ ]:
print(ef_bf1n2_s_us[1][ef_bf1n2_s_us[1].obs_names.isin(ef_bf1n2_s_us[1].obs_names[1:10].to_list()),ef_bf1n2_s_us[1].var_names.isin(ef_bf1n2_s_us[1].var_names[1:15].to_list())].X)

Assignment of variables for the number of PCs and number of neighbours to use in calculating neighbours using UMAP as the default or sklearn and other methods
The PCs and number of neighbours are also used to calculate first&second order moments for each cell across it's neighbours for spliced and unspliced counts

This parameters can be played with to adjust visualization but we are using the default for both which is 30

We use UMAPs calculated from seurat. So this step is not that key but to use scv generated UMAP follow there are some instructions here https://github.com/theislab/scvelo/issues/112#issuecomment-1034925572

In [ ]:
bf1_pcs = [6, 6]

bf1_neighbours = [30, 30]


### Filtering and calculating moments for recovery of dynamics and RNA velocity 

Defining a function to perform: 

Filtering based on the minimum number of spliced and unspliced counts required for a gene (20 here. i.e we only want genes with atleast 20 spliced counts and 20 unspliced counts across all cells).
Calculate PCs, neighbours, and moments

scv.pp.moments returns 
 - Ms (.layers) – dense matrix with first order moments of spliced counts.
 - Mu (.layers) – dense matrix with first order moments of unspliced counts.


In [ ]:
def velo_calc(subset_ad, nns, npcs):
    scv.pp.filter_and_normalize(subset_ad, min_shared_counts=20, n_top_genes=3000)
    # print(subset_ad[1:2,subset_ad.var_names.to_list()[1:10]].X)
    sc.tl.pca(subset_ad)
    sc.pp.neighbors(subset_ad, n_pcs=npcs, n_neighbors=nns)
    scv.pp.moments(subset_ad, n_pcs=None, n_neighbors=None)
    # print(subset_ad[1:3,subset_ad.var_names.to_list()[1:10]].layers["spliced"])
    return subset_ad


In [ ]:
for obj, nns, npcs in zip(ef_bf1n2_s_us, bf1_neighbours, bf1_pcs):
    obj = velo_calc(subset_ad = obj, nns=nns, npcs=npcs)

In [ ]:
ef_bf1n2_s_us

## Assessing metrics used in RNA velocity calculaiton and formating gene names to human readable

Saving the contents of var as csv

In [ ]:
# for obj1, day in zip(ef_qcd_bf1_scv, v3_days):
#     obj1.var.to_csv("data/processed/PfDB52e/day5/sone_ksc/ef_qcd_bf1_" + day + ".csv")


In [ ]:
# for obj2, day in zip(ef_bf1n2_s_us, v3_days_fnl):
#     obj2.var.to_csv("data/processed/PfDB52e/day5/sone_ksc/ef_qcd_bf2_" + day + ".csv")

In [ ]:
# ef_qcd_bf1_d5_scv.var.to_csv("pfvelo.csv")

In [ ]:
sc.pl.umap(ef_bf1n2_s_us[0], color="stage_bfs", components='2,3')

In [ ]:
sc.pl.umap(ef_bf1n2_s_us[1], color="stage_bfs", components='2,3')

Reading in plasmoDB data to merge with scv objects so as to get human readable gene names

In [ ]:
Pf3D7_genes_plasmoDB = pd.read_csv("data/raw/Pf3d7_Genes_Summary_190322.csv")

In [ ]:
Pf3D7_genes_plasmoDB.columns

Rename column name "Gene Name or Symbol" to "Gene" 

In [ ]:
Pf3D7_genes_plasmoDB.rename(columns={"Gene Name or Symbol" : "Gene"}, inplace=True)

Set 'Gene ID' as index in preparation for merging with var DF in scvelo anndata object

In [ ]:
Pf3D7_genes_plasmoDB.set_index(keys = "Gene ID", drop=False, append=False, inplace=True, verify_integrity=False)

Remove rows with duplicated index (ie duplicated 'Gene ID')

In [ ]:
Pf3D7_genes_plasmoDB = Pf3D7_genes_plasmoDB[~Pf3D7_genes_plasmoDB.index.duplicated(keep='first')]

Replacing the NaNs in 'Gene' with pf 'Gene ID' ie the table index

In [ ]:
nan_strings = ["{}".format(i) for i in Pf3D7_genes_plasmoDB['Gene ID']]

In [ ]:
Pf_genes_noNA=pd.DataFrame(Pf3D7_genes_plasmoDB['Gene']).apply(lambda c: c.where(c.notnull(), nan_strings))

In [ ]:
from socket import socket

Sorting duplicates in 'Gene' such as VAR, RIF, AP2 by merging with 'Gene ID' to get all the gene IDs to be unique

In [ ]:
gene_id_name = [m+'_'+str(n) for m,n in zip(Pf3D7_genes_plasmoDB['Gene ID'], Pf3D7_genes_plasmoDB['Gene'])]

In [ ]:
Pf_genes_noNA_noDup=Pf_genes_noNA.apply(lambda c: c.where(~c.duplicated(), gene_id_name)).drop_duplicates(keep='first')

In [ ]:
Pf_genes_noNA_noDup

Merge Pf_genes_noNA_noDup which is an offshoot of Pf3D7_genes_plasmoDB that has formatted 'Gene' names with its parent table to keep only important variables

Need to check to ensure that the was proper copying at the and not just referencing as is common python behavior

In [ ]:
Pf_genes_cln = pd.merge(Pf_genes_noNA_noDup, Pf3D7_genes_plasmoDB[["source_id", "Gene ID", "NonSyn/Syn SNP Ratio All Strains", "# Exons in Gene", 'Annotated 3\' UTR length', 'Annotated 5\' UTR length', 'Gene Strand', 'Transcript Length', 'Computed GO Functions', 'Computed GO Components', 'Computed GO Processes', 'Curated GO Components', 'Curated GO Functions', 'Curated GO Processes']], how='left', left_index = True, right_index = True)

In [ ]:
pd.DataFrame(Pf_genes_cln)

In [ ]:
# pd.DataFrame(Pf_genes_cln)['Annotated 5\' UTR length']

Merging the formatted PlasmoDB gene names and other metadata with the var DF in the anndata and replacing any NAs in the Gene name with the var index which is the gene ID. 

In [ ]:
##Incorporate Pf gene names
# Pf3D7_gene_list_nonNA_bf1 = []
for obj1 in ef_bf1n2_s_us:
    obj1.var = pd.merge(obj1.var, Pf_genes_cln[["source_id", "Gene",  "Gene ID", "NonSyn/Syn SNP Ratio All Strains", "# Exons in Gene", 'Annotated 3\' UTR length', 'Annotated 5\' UTR length', 'Gene Strand', 'Transcript Length', 'Computed GO Functions', 'Computed GO Components', 'Computed GO Processes', 'Curated GO Components', 'Curated GO Functions', 'Curated GO Processes']], how='left', left_index = True, right_index = True)
    ##Creating a vector of gene ids to use in stead of genes with no gene names
    nan_strings = ["{}".format(i) for i in obj1.var.index]
    obj1.var['Gene'] = pd.DataFrame(obj1.var.Gene).apply(lambda c: c.where(c.notnull(), nan_strings))
    

Checking for duplicates in the Gene column of var

In [ ]:
for obj1 in ef_bf1n2_s_us:
    print(pd.DataFrame(obj1.var.Gene).duplicated().any())


setting Gene column in var DF to index and removing the index name

Adding QC metrics for unspliced, spliced and ambigous counts

In [ ]:
for obj in ef_bf1n2_s_us:
    sc.pp.calculate_qc_metrics(obj, 
                               expr_type='unspliced', 
                               var_type='unspliced', 
                               percent_top=(50, 100), 
                               layer='unspliced', 
                               inplace=True, 
                               log1p=True)
    sc.pp.calculate_qc_metrics(obj, 
                               expr_type='spliced', 
                               var_type='spliced', 
                               percent_top=(50, 100), 
                               layer='spliced', 
                               inplace=True, 
                               log1p=True)
    sc.pp.calculate_qc_metrics(obj, 
                               expr_type='ambiguous', 
                               var_type='ambiguous', 
                               percent_top=(50, 100), 
                               layer='ambiguous', 
                               inplace=True, 
                               log1p=True)

In [ ]:
ef_bf1n2_s_us[1].var.columns

In [ ]:
import seaborn as sns

In [ ]:
sns.jointplot(
    data=ef_bf1n2_s_us[1].var,
    x="log1p_total_spliced",
    y="log1p_total_unspliced",
    kind="hex",
)

In [ ]:
sns.scatterplot(
    data=ef_bf1n2_s_us[1].var,
    x="total_spliced",
    y="total_unspliced",
    # kind="hex",
)

In [ ]:
sns.jointplot(
    data=ef_bf1n2_s_us[1].var,
    x="# Exons in Gene",
    y="total_unspliced",
    kind="hex",
)

In [ ]:
sns.scatterplot(
    data=ef_bf1n2_s_us[1].var,
    x="# Exons in Gene",
    y="log1p_total_unspliced",
    # kind="hex",
)

In [ ]:
sns.scatterplot(
    data=ef_bf1n2_s_us[1].var,
    x="Annotated 3' UTR length",
    y="total_unspliced",
    # kind="hex",
)

In [ ]:
sns.scatterplot(
    data=ef_bf1n2_s_us[1].var,
    x="Annotated 5' UTR length",
    y="total_unspliced",
    # kind="hex",
)

In [ ]:
sns.scatterplot(
    data=ef_bf1n2_s_us[1].var,
    x="Transcript Length",
    y="total_unspliced",
    # kind="hex",
)

In [ ]:
sns.scatterplot(
    data=ef_bf1n2_s_us[1].var,
    x="NonSyn/Syn SNP Ratio All Strains",
    y="total_unspliced",
    # kind="hex",
)

In [ ]:
sns.scatterplot(
    data=ef_bf1n2_s_us[1].var,
    x="Transcript Length",
    y="total_unspliced",
    # kind="hex",
)

Plotting correlation of spliced and unspliced counts

In [ ]:
for obj1 in ef_bf1n2_s_us:
    sc.pl.scatter(obj, x= 'initial_size_spliced', y='initial_size_unspliced')

In [ ]:
ef_bf1n2_s_us[1].var

Plotting the spliced unspliced distribution

In [ ]:
for obj2 in ef_bf1n2_s_us:
    sc.pl.violin(obj2, ['initial_size_spliced', 'initial_size_unspliced', 'initial_size'], jitter=0.4, multi_panel=True)

In [ ]:
#############      JUMP TO scv.tl.recover_dynamics()################

Check scvelo version

In [ ]:
scv.logging.print_version()

In [ ]:
#############      JUMP here scv.tl.recover_dynamics()################

Plotting proportion of spliced and unspliced counts

In [ ]:
v3_days_fnl = ['bf1','bf2']

In [ ]:
for obj2, v3_day in zip(ef_bf1n2_s_us, v3_days_fnl):
    scv.pl.proportions(obj2, groupby="stage_bfs", save="plots/"+v3_day+"_bf2_splices_unsplices_props.png")


### RNA velocity - Stochastic

In [ ]:
ef_scv_stochastic=[]
for obj in ef_bf1n2_s_us:
    ef_scv_stochastic.append(obj.copy())

In [ ]:
ef_bf1n2_s_us

In [ ]:
for obj1 in ef_scv_stochastic:
    scv.tl.velocity(obj1, mode='stochastic', n_jobs=5, min_r2=0.2, perc = [20, 80])
    scv.tl.velocity_graph(obj1)

In [ ]:
for obj in ef_scv_stochastic:
    scv.pl.velocity_embedding_stream(obj, basis='umap', projection="3d", components='2,3', color='stage_bfs')
    # scv.pl.velocity_embedding(obj, basis='umap', components='2,3', arrow_length=2, arrow_size=2)
    scv.pl.velocity_embedding_grid(obj, basis='umap', components='2,3', arrow_length=2, arrow_size=2)

### RNA velocity - Dynamical

In [ ]:
ef_scv_dynamical=[]
for obj in ef_bf1n2_s_us:
    ef_scv_dynamical.append(obj.copy())

In [ ]:
for obj in ef_scv_dynamical:
    scv.tl.recover_dynamics(obj, n_jobs=8, max_iter=20, use_raw=True)

In [ ]:
ef_scv_dynamical[1].obs['orig.ident']

In [ ]:
for obj in ef_scv_dynamical:
    scv.tl.velocity(obj, mode='dynamical', n_jobs=5, groupby='orig.ident', min_likelihood=0.05, min_r2=0.2)
    scv.tl.velocity_graph(obj)

In [ ]:
for obj in ef_scv_dynamical:
    scv.pl.velocity_embedding_stream(obj, basis='umap', projection="3d", components='1,3', color='stage_bfs')
    # scv.pl.velocity_embedding(obj, basis='umap', components='2,3', arrow_length=2, arrow_size=2)
    scv.pl.velocity_embedding_grid(obj, basis='umap', components='2,3', arrow_length=2, arrow_size=2)

# Driver genes at specific clusters

## Pseudotime cellrank estimation

Recovering all splicing kinetics using expectation maximization likelihood but estimating velocity genes using steady state model if the 'velocity_genes' column not present in var. ('If var_names=’velocity_genes’ but there is no column ‘velocity_genes’ in adata.var, velocity genes are estimated using the steady state model').
More details on estimation estimation of velocity genes here https://github.com/theislab/scvelo/issues/497

Transcription rate (alpha), splicing rate (beta), degradation rate (gamma), switching timepoints, latent time (cell's internal clock), likelihood of model fit and scaling factors etc are iteratively inferred by maximizing joint likelihood.

The option use_raw = True can be used to direct the use of raw spliced (.layers[‘sliced’]) counts to be used in estimating the dynamics as opposed to moments in .layers[‘Ms’]. this seems to be recomended here https://www.biorxiv.org/content/10.1101/2022.02.12.480214v1

In [ ]:
ef_scv_PK=[]
for obj in ef_bf1n2_s_us:
    ef_scv_PK.append(obj.copy())

In [ ]:
ef_scv_PK[1].obs.pseudo_mg

In [ ]:
plt.rcParams["figure.figsize"] = (7, 5)
for i, obj in enumerate(ef_scv_PK):
    scv.pl.scatter(
        obj,
        c="pseudo_mg",
        basis="umap",
        legend_loc="right",
        color_map="gnuplot2",
        components='2,3'
)
    plt.rcParams["figure.figsize"] = (7, 5)

In [ ]:
# for obj, v3_day in zip(ef_scv_PK, v3_days_all_fnl):
#     scv.pl.velocity_embedding_stream(
#     adata, color="Stage", vkey="T_fwd", basis="force_directed", legend_loc="right"
# )

End - Kernels and estimators

In [ ]:
# for obj, v3_day in zip(ef_scv_PK, v3_days_all_fnl):
#     scv.pl.velocity_embedding_stream(obj, basis='umap', projection="3d", components='2,3', color='stage_bfs')
#     # scv.pl.velocity_embedding(obj, basis='umap', components='2,3', arrow_length=2, arrow_size=2)
#     scv.pl.velocity_embedding_grid(obj, basis='umap', components='2,3', arrow_length=2, arrow_size=2)

In [ ]:
asx_gam_cols = ["#ECE2F0", "#D0D1E6", "#6BAED6", "#4292C6", "#2171B5", "#FDAE6B", "#E6550D", "#A1D99B", "#31A354", "#02818A", "#016C59"]

In [ ]:
asx_gam_cols_dict = dict(zip(ef_scv_PK[0].obs.stage_bfs.cat.categories.tolist(), asx_gam_cols))

In [ ]:
ef_scv_PK[1].obs.value_counts('stage_bfs')

In [ ]:
# scv.pl.velocity_embedding_stream(ef_scv_PK[0], basis='umap', projection="3d", components='2,3', color='stage_bfs', palette=asx_gam_cols_dict)

In [ ]:
# scv.pl.velocity_embedding_stream(ef_scv_PK[0], basis='umap', projection="3d", components='2,3', color='stage_bfs', palette=asx_gam_cols_dict, add_outline=['early trophozoite', 'Gametocytes (developing)','Gametocytes (branching)','Gametocytes (early male)','Gametocytes (late male)','Gametocytes (early female)','Gametocytes (late female)', 'early schizont'])
    # scv.pl.velocity_embedding(obj, basis='umap', components='2,3', arrow_length=2, arrow_size=2)
# scv.pl.velocity_embedding_grid(ef_scv_PK_lbld, basis='umap', components='2,3', arrow_length=2, arrow_size=2)

In [ ]:
# scv.pl.velocity_embedding_stream(ef_scv_PK[0], basis='umap', projection="3d", components='2,3', color='stage_bfs', palette=asx_gam_cols_dict, add_outline=['early trophozoite', 'late ring', 'Gametocytes (developing)', 'early schizont'])
    # scv.pl.velocity_embedding(obj, basis='umap', components='2,3', arrow_length=2, arrow_size=2)
# scv.pl.velocity_embedding_grid(ef_scv_PK_lbld, basis='umap', components='2,3', arrow_length=2, arrow_size=2)

In [ ]:
# scv.pl.velocity_embedding_stream(ef_scv_PK[0], basis='umap', projection="3d", components='2,3', color='stage_bfs', palette=asx_gam_cols_dict, add_outline=['Gametocytes (committed)', 'Gametocytes (developing)', 'Gametocytes (late male)', 'Gametocytes (late female)'])
    # scv.pl.velocity_embedding(obj, basis='umap', components='2,3', arrow_length=2, arrow_size=2)
# scv.pl.velocity_embedding_grid(ef_scv_PK_lbld, basis='umap', components='2,3', arrow_length=2, arrow_size=2)

In [ ]:
# scv.pl.velocity_embedding_stream(ef_scv_PK[0], basis='umap', projection="3d", components='2,3', color='stage_bfs', palette=asx_gam_cols_dict, add_outline=['Gametocytes (developing)','Gametocytes (branching)', 'Gametocytes (late male)', 'Gametocytes (late female)'],size=20, fontsize=1)
    # scv.pl.velocity_embedding(obj, basis='umap', components='2,3', arrow_length=2, arrow_size=2)
# scv.pl.velocity_embedding_grid(ef_scv_PK_lbld, basis='umap', components='2,3', arrow_length=2, arrow_size=2)

### Cellrank

Start - Kernels and estimators 

Low level Kernels and estimators

In [ ]:
ptk_bf = []
for obj in ef_scv_PK:
    ptk_bf.append(PseudotimeKernel(obj, time_key='pseudo_mg').compute_transition_matrix())
    
    # x = VelocityKernel(obj).compute_transition_matrix()
    # vk_bf.append(x)
    # # cr.tl.terminal_states(obj, cluster_key="Stage", weight_connectivities=0.2)

In [ ]:
ptk_g_bf = []
for ptk in ptk_bf:
    x = cr.tl.estimators.GPCCA(ptk)
    ptk_g_bf.append(x)
    

In [ ]:
ptk_g_bf

In [ ]:
for g in ptk_g_bf:
    g.compute_schur(n_components=20)
    g.plot_spectrum()

In [ ]:
# for g in ptk_g_bf2:
#     g.compute_schur(n_components=20)
#     g.plot_spectrum()


In [ ]:
for i, g in enumerate(ptk_g_bf):
    if i == 0:
        g.compute_macrostates(n_states=5, cluster_key="stage_bfs")
        g.plot_macrostates(components = '2,3')
    else:
        g.compute_macrostates(n_states=3, cluster_key="stage_bfs")
        g.plot_macrostates(components = '2,3')


Manually setting terminal states
Combining late males and early males as terminal state for day 5 and using late male for all the other timepoints
Using Early female as terminal state for day 5 and using late female for all the other days

In [ ]:
sc.pl.umap(ef_scv_PK[0], color='stage_bfs', components='2,3')

In [ ]:
ef_scv_PK[1].obs.value_counts(['stage_bfs'])

In [ ]:
# for i,g in enumerate(ptk_g_bf):
#     if i == 0:
#         g.set_terminal_states_from_macrostates(names= {"Gametocytes (developing)":"Gametocyte", "late trophozoite":"Asexual"}, n_cells=1700) 
#     else:
#         g.set_terminal_states_from_macrostates(names= {"gametocyte (male)":"Male", "gametocyte (female)":"Female"}, n_cells=1700) 


In [ ]:
for i,g in enumerate(ptk_g_bf):
    if i == 0:
        g.set_terminal_states_from_macrostates(names= {"Gametocytes (developing)":"Gametocyte", "late trophozoite":"Asexual"}, n_cells=900) 
    else:
        g.set_terminal_states_from_macrostates(names= {"Gametocytes (late male)":"Male", "Gametocytes (late female)":"Female"}, n_cells=900) 


In [ ]:
# # ef_scv_PK[0][ef_scv_PK[0].obs["stage_bfs"].isin(["Gametocytes (developing)","Gametocytes (branching)","Gametocytes (early male)","Gametocytes (late male)","Gametocytes (early female)","Gametocytes (late female)" ])].obs.Stage
# ef_scv_PK[0][ef_scv_PK[0].obs["stage_bfs"].isin(["Gametocytes (developing)","Gametocytes (branching)","Gametocytes (early male)","Gametocytes (late male)","Gametocytes (early female)","Gametocytes (late female)" ])].obs.Stage
#                                   obj[obj.obs["stage_bfs"].isin(["Gametocytes (developing)","Gametocytes (branching)","Gametocytes (early male)","Gametocytes (late male)","Gametocytes (early female)","Gametocytes (late female)" ])].obs_names, "Early schizont": obj[obj.obs["stage_bfs"] == "early schizont"].obs_names})

### Early gam and asexual driver

In [ ]:
for g in ptk_g_bf:
    g.compute_absorption_probabilities()

In [ ]:
for g in ptk_g_bf:
    g.plot_absorption_probabilities(components ='2,3')

In [ ]:
# for i,g in enumerate(ptk_g_bf):
#     if i == 0:
#         x = g.compute_lineage_drivers(lineages="Gametocyte", return_drivers=True, cluster_key = 'stage_bfs', clusters= ['Gametocytes (committed)', 'early trophozoite'])
#         early_gam_drivers.append(x)
        
#         g.set_terminal_states_from_macrostates(names= {"Gametocytes (developing)":"Gametocyte", "late trophozoite":"Asexual"}, n_cells=1700) 
#     else:
#         x = g.compute_lineage_drivers(lineages="Late female", return_drivers=True, cluster_key = 'stage_bfs', clusters= ['Gametocytes (developing)'])
#         early_gam_drivers.append(x)
        
#         g.set_terminal_states_from_macrostates(names= {"gametocyte (male)":"Late male", "gametocyte (female)":"Late female"}, n_cells=1700) 


In [ ]:
ptk_g_bf_dr=[]
ptk_g_bf_dr.append(ptk_g_bf[0])
ptk_g_bf_dr.append(ptk_g_bf[0])

In [ ]:
len(early_gam_drivers)

In [ ]:
early_gam_drivers = []
for i, g in enumerate(ptk_g_bf_dr):
    if i == 0:
        early_gam_drivers.append(g.compute_lineage_drivers(lineages="Gametocyte", return_drivers=True, cluster_key = 'stage_bfs', clusters= ['late ring','early trophozoite','Gametocytes (committed)']))
    else:
        early_gam_drivers.append(g.compute_lineage_drivers(lineages="Gametocyte", return_drivers=True))

In [ ]:
for drvs in early_gam_drivers:
    print(drvs.index.tolist()[:10])

In [ ]:
for g in ptk_g_bf_dr:
    g.plot_lineage_drivers("Gametocyte", n_genes=4, components='2,3')

Computing driver genes in the gametocyte(branching) cluster that are driving cells to the male cluster

Plotting top 4 driver genes per timepoint

Upset plots to assess driver genes in branching gams that is shared between day10 new and day 10 old datasets and the 2 datasets combined using retaining only genes positively correlated to absorption probabilities towards terminal males where P < 0.001 as qvalue is NA https://github.com/theislab/cellrank/issues/819

Phase potraits highlighting behaviour of gene in 'Gametocytes (branching)' cluster. We should start seeing activation atleast in cells that will then on become female.

In [ ]:
for i, drvs in enumerate(early_gam_drivers):
    scv.pl.scatter(ef_scv_PK[0], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['late ring','early trophozoite','Gametocytes (committed)'])
    scv.pl.scatter(ef_scv_PK[0], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (developing)'])
    scv.pl.scatter(ef_scv_PK[0], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['late trophozoite'])

Phase potraits highlighting behaviour of gene in 'Gametocytes (early male)','Gametocytes (late male)' cluster. We should see activation

Summary table of the female driver genes in the 'Gametocytes (committed and developing)' for all timepoints

In [ ]:
early_gam_drivers_bf1_pk=[]
for drvs in early_gam_drivers:
    early_gam_drivers_bf1_pk.append(pd.merge(drvs, Pf_genes_cln, how='left', left_index=True, right_index=True))

In [ ]:
corr_clust = ['bf1', 'all']

In [ ]:
for drvs, cc in zip(early_gam_drivers_bf1_pk, corr_clust):
    drvs.to_csv(processed_data + '../RoadtoUMAP/early_gam_drivers_bf1_pk_' + cc +'.csv')

#### Asexual drivers

In [ ]:
asexual_drivers = []
for i, g in enumerate(ptk_g_bf_dr):
    if i == 0:
        asexual_drivers.append(g.compute_lineage_drivers(lineages="Asexual", return_drivers=True, cluster_key = 'stage_bfs', clusters= ['late ring','early trophozoite','Gametocytes (committed)']))
    else:
        asexual_drivers.append(g.compute_lineage_drivers(lineages="Asexual", return_drivers=True))

In [ ]:
asexual_drivers[1]

In [ ]:
for drvs in asexual_drivers:
    print(drvs.index.tolist()[:10])

In [ ]:
for g in ptk_g_bf_dr:
    g.plot_lineage_drivers("Asexual", n_genes=4, components='2,3')

In [ ]:
for i, drvs in enumerate(asexual_drivers):
    scv.pl.scatter(ef_scv_PK[0], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['late ring','early trophozoite','Gametocytes (committed)'])
    scv.pl.scatter(ef_scv_PK[0], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['late trophozoite'])
    scv.pl.scatter(ef_scv_PK[0], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (developing)'])

In [ ]:
asexual_drivers_bf1_pk=[]
for drvs in asexual_drivers:
    asexual_drivers_bf1_pk.append(pd.merge(drvs, Pf_genes_cln, how='left', left_index=True, right_index=True))

In [ ]:
for drvs, cc in zip(asexual_drivers_bf1_pk, corr_clust):
    drvs.to_csv(processed_data + '../RoadtoUMAP/asexual_drivers_bf1_pk_' + cc +'.csv')

In [ ]:
asexual_drivers_bf1_pk[1]

In [ ]:
sc.pl.umap(ef_scv_PK[0], color='assignment2')

### Male female driver analysis

Assessing genes involved in fate determination towards the male and female clusters with higher resolution using chunks generated by pseudotime

Manually setting terminal states
Combining late males and early males as terminal state for day 5 and using late male for all the other timepoints
Using Early female as terminal state for day 5 and using late female for all the other days

In [ ]:
for g in ptk_g_bf:
    g.plot_absorption_probabilities(components='2,3')

In [ ]:
ef_scv_PK[0].obs.clusters_gradients

### Computing driver genes in the gametocyte(branching) cluster that are driving cells to the male and female cluster

#### To the male

In [ ]:
ptk_g_bf2_dr=[]
ptk_g_bf2_dr.append(ptk_g_bf[1])
ptk_g_bf2_dr.append(ptk_g_bf[1])

In [ ]:
male_drivers = []
for i, g in enumerate(ptk_g_bf2_dr):
    if i == 0:
        male_drivers.append(g.compute_lineage_drivers(lineages="Male", return_drivers=True, cluster_key = 'stage_bfs', clusters= ['Gametocytes (developing)', 'Gametocytes (branching)']))
    else:
        male_drivers.append(g.compute_lineage_drivers(lineages="Male", return_drivers=True))

In [ ]:
# male_drivers = ptk_g_bf[1].compute_lineage_drivers(lineages="Male", return_drivers=True, cluster_key = 'stage_bfs', clusters= ['Gametocytes (developing)', 'Gametocytes (branching)'])

In [ ]:
male_drivers[1]

In [ ]:
for drvs in male_drivers:
    print(drvs.index.tolist()[:10])

Plotting top 4 driver genes per timepoint

In [ ]:
for g in ptk_g_bf2_dr:
    g.plot_lineage_drivers("Male", n_genes=4, components='2,3') 

Phase potraits highlighting behaviour of gene in 'Gametocytes (branching)' cluster. We should start seeing activation atleast in cells that will then on become female.

Phase potraits highlighting behaviour of gene in 'Gametocytes (early male)','Gametocytes (late male)' cluster. We should see activation

In [ ]:
for i, drvs in enumerate(male_drivers):
    scv.pl.scatter(ef_scv_PK[1], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (developing)','Gametocytes (branching)'])
    scv.pl.scatter(ef_scv_PK[1], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (early male)','Gametocytes (late male)'])
    scv.pl.scatter(ef_scv_PK[1], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (early female)','Gametocytes (late female)'])

Upset plots to assess driver genes in branching gams that is shared between day10 new and day 10 old datasets and the 2 datasets combined using retaining only genes positively correlated to absorption probabilities towards terminal males where P < 0.001 as qvalue is NA https://github.com/theislab/cellrank/issues/819

In [ ]:
for i, drvs in enumerate(male_drivers):
    scv.pl.umap(ef_scv_PK[1],  layer='Mu',  color=drvs.index[0:5].to_list(), components='2,3')

In [ ]:
male_drivers_bf2_pk=[]
for drvs in male_drivers:
    male_drivers_bf2_pk.append(pd.merge(drvs, Pf_genes_cln, how='left', left_index=True, right_index=True))

In [ ]:
for drvs, cc in zip(male_drivers_bf2_pk, corr_clust):
    drvs.to_csv(processed_data + '../RoadtoUMAP/male_drivers_bf2_pk_' + cc +'.csv')

In [ ]:
# Pf_genes_cln.loc[Pf_genes_cln.Gene.isin(branching_gam_drivers_m[0].index[:5].to_list())]

#### To the female cluster

In [ ]:
# female_drivers = ptk_g_bf[1].compute_lineage_drivers(lineages="Female", return_drivers=True, cluster_key = 'stage_bfs', clusters= ['Gametocytes (developing)', 'Gametocytes (branching)'])

In [ ]:
female_drivers = []
for i, g in enumerate(ptk_g_bf2_dr):
    if i == 0:
        female_drivers.append(g.compute_lineage_drivers(lineages="Female", return_drivers=True, cluster_key = 'stage_bfs', clusters= ['Gametocytes (developing)', 'Gametocytes (branching)']))
    else:
        female_drivers.append(g.compute_lineage_drivers(lineages="Female", return_drivers=True))


In [ ]:
female_drivers[1]

In [ ]:
for drvs in female_drivers:
    print(drvs.index.tolist()[:10])

Plotting top 4 driver genes per timepoint

In [ ]:
ptk_g_bf2_dr[1]

In [ ]:
for g in ptk_g_bf2_dr:
    g.plot_lineage_drivers("Female", n_genes=4, components='2,3') 

Phase potraits highlighting behaviour of gene in 'Gametocytes (branching)' cluster. We should start seeing activation atleast in cells that will then on become female.

Phase potraits highlighting behaviour of gene in 'Gametocytes (early male)','Gametocytes (late male)' cluster. We should see activation

In [ ]:
for i, drvs in enumerate(female_drivers):
    scv.pl.scatter(ef_scv_PK[1], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (developing)','Gametocytes (branching)'])
    scv.pl.scatter(ef_scv_PK[1], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (early female)','Gametocytes (late female)'])
    scv.pl.scatter(ef_scv_PK[1], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (early male)','Gametocytes (late male)'])

Upset plots to assess driver genes in branching gams that is shared between day10 new and day 10 old datasets and the 2 datasets combined using retaining only genes positively correlated to absorption probabilities towards terminal males where P < 0.001 as qvalue is NA https://github.com/theislab/cellrank/issues/819

In [ ]:
for i, drvs in enumerate(female_drivers):
    scv.pl.umap(ef_scv_PK[1],  layer='Mu',  color=drvs.index[0:5].to_list(), components='2,3')

In [ ]:
female_drivers_bf2_pk=[]
for drvs in female_drivers:
    female_drivers_bf2_pk.append(pd.merge(drvs, Pf_genes_cln, how='left', left_index=True, right_index=True))

In [ ]:
for drvs, cc in zip(female_drivers_bf2_pk, corr_clust):
    drvs.to_csv(processed_data + '../RoadtoUMAP/female_drivers_bf2_pk_' + cc +'.csv')

Upset plot for female

Upset plots to assess driver genes in branching gams that is shared between day10 new and day 10 old datasets and the 2 datasets combined using retaining only genes positively correlated to absorption probabilities towards terminal males where P < 0.001 as qvalue is NA. bug in scvelo https://github.com/theislab/cellrank/issues/819

Fitting GAMs for assesment of gene expression clusters over latent time

In [ ]:
modelGAM_male_lineage = []
for obj in ef_scv_PK:
    model=cr.ul.models.GAM(obj)
    modelGAM_male_lineage.append(model)


In [ ]:
for i, obj in enumerate(ef_scv_PK):
    cr.pl.cluster_lineage(
            obj,
            model = modelGAM_male_lineage[i],
            genes=branching_gam_drivers_m[i].index[(branching_gam_drivers_m[i]['Late male_qval'] < 0.001) & (branching_gam_drivers_m[i]['Late male_corr'] > 0.2)].to_list(),
            lineage="Late male",
            time_key="pseudo_mg",
            show_progress_bar=False,
            recompute=True,
            sharey=True,
            covariate_key=['stage_bfs']
        )


In [ ]:
for i, obj in enumerate(ef_scv_PK):
    cr.pl.cluster_lineage(
            obj,
            model = modelGAM_male_lineage[i],
            genes=branching_gam_drivers_m[i].index[(branching_gam_drivers_m[i]['Late male_qval'] < 0.001) & (branching_gam_drivers_m[i]['Late male_corr'] < -0.2)].to_list(),
            lineage="Late male",
            time_key="pseudo_mg",
            show_progress_bar=False,
            recompute=True,
            sharey=True,
            covariate_key=['stage_bfs']
        )

In [ ]:
v3_days_all_fnl

In [ ]:
# plt.rcParams["figure.figsize"] = (120, 5)
for i, obj in enumerate(ef_scv_PK):
    cr.pl.gene_trends(
        obj,
        lineages=['Late female','Late male'],
        model=modelGAM_male_lineage[i],
        data_key="X",
        genes=branching_gam_drivers_m[i].index[(branching_gam_drivers_m[i]['Late male_qval'] < 0.0000000000001) & (branching_gam_drivers_m[i]['Late male_corr'] < -0.3)].to_list(),
        ncols=5,
        time_key="pseudo_mg",
        same_plot=True,
        hide_cells=True,
        figsize=(50, 6),
        n_test_points=200)
    
# plt.rcParams["figure.figsize"] = (7, 5)

In [ ]:
modelGAM_female_lineage_bf2 = []
for obj in ef_scv_PK:
    model=cr.ul.models.GAM(obj)
    modelGAM_female_lineage_bf2.append(model)


In [ ]:
modelGAM_female_lineage_bf2 = []
for obj in ef_scv_PK:
    model=cr.ul.models.GAM(obj)
    modelGAM_female_lineage_bf2.append(model)

Driver genes in CD positively correlated with progression to female fate

Driver genes in gambranch positively correlated with progression to female fate

In [ ]:
for i, obj in enumerate(ef_scv_PK):
    cr.pl.cluster_lineage(
            obj,
            model = modelGAM_female_lineage_bf2[i],
            genes=branching_gam_drivers_f[i].index[(branching_gam_drivers_f[i]['Late female_qval'] < 0.0000000000001) & (branching_gam_drivers_f[i]['Late female_corr'] > -0.2)].to_list(),
            lineage="Late female",
            time_key="pseudo_mg",
            show_progress_bar=False,
            recompute=True,
            sharey=True,
            covariate_key=['stage_bfs']
        )

branching_gam_drivers_f is the equivalent of female_drivers_CDB for the branching driver list. Care should be taken to avoid being confused by names

## Rvelo cellrank

In [ ]:
vtk_bf1n2 = []
for obj in ef_scv_stochastic:
    vtk_bf1n2.append(VelocityKernel(obj).compute_transition_matrix())
    

In [ ]:
ck_bf1n2 = []
for obj in ef_scv_stochastic:
    x = ConnectivityKernel(obj).compute_transition_matrix()
    ck_bf1n2.append(x)

In [ ]:
combined_kernel_bf1n2 = []
for obj, vk, ck in zip(ef_scv_stochastic, vtk_bf1n2, ck_bf1n2):
    x = 0.5 * vk + 0.5 * ck
    combined_kernel_bf1n2.append(x)

In [ ]:
print(combined_kernel_bf1n2)

In [ ]:
vtk_g_bf1n2 = []
for ck in combined_kernel_bf1n2:
    x = GPCCA(ck)
    vtk_g_bf1n2.append(x)
    

In [ ]:
# vtk_g_bf1n2 = []
# for ptk in vtk_bf1n2:
#     x = cr.tl.estimators.GPCCA(ptk)
#     vtk_g_bf1n2.append(x)
    

In [ ]:
for g in vtk_g_bf1n2:
    g.compute_schur(n_components=20)
    g.plot_spectrum()

In [ ]:
# for g in vtk_g_bf1n22:
#     g.compute_schur(n_components=20)
#     g.plot_spectrum()


In [ ]:
for i, g in enumerate(vtk_g_bf1n2):
    if i == 0:
        g.compute_macrostates(n_states=4, cluster_key="stage_bfs")
        g.plot_macrostates(components = '2,3')
    else:
        g.compute_macrostates(n_states=4, cluster_key="stage_bfs")
        g.plot_macrostates(components = '2,3')


Manually setting terminal states
Combining late males and early males as terminal state for day 5 and using late male for all the other timepoints
Using Early female as terminal state for day 5 and using late female for all the other days

In [ ]:
ef_scv_stochastic[0].obs.value_counts(['stage_bfs'])

In [ ]:
# for i,g in enumerate(vtk_g_bf1n2):
#     if i == 0:
#         g.set_terminal_states_from_macrostates(names= {"Gametocytes (developing)":"Gametocyte", "late trophozoite":"Asexual"}, n_cells=1700) 
#     else:
#         g.set_terminal_states_from_macrostates(names= {"gametocyte (male)":"Male", "gametocyte (female)":"Female"}, n_cells=1700) 


In [ ]:
for i,g in enumerate(vtk_g_bf1n2):
    if i == 0:
        g.set_terminal_states_from_macrostates(names= {"Gametocytes (developing)":"Gametocyte", "late trophozoite":"Asexual"}, n_cells=900) 
    else:
        g.set_terminal_states_from_macrostates(names= {"Gametocytes (late male)":"Male", "Gametocytes (late female)":"Female"}, n_cells=900) 


In [ ]:
# # ef_scv_stochastic[0][ef_scv_stochastic[0].obs["stage_bfs"].isin(["Gametocytes (developing)","Gametocytes (branching)","Gametocytes (early male)","Gametocytes (late male)","Gametocytes (early female)","Gametocytes (late female)" ])].obs.Stage
# ef_scv_stochastic[0][ef_scv_stochastic[0].obs["stage_bfs"].isin(["Gametocytes (developing)","Gametocytes (branching)","Gametocytes (early male)","Gametocytes (late male)","Gametocytes (early female)","Gametocytes (late female)" ])].obs.Stage
#                                   obj[obj.obs["stage_bfs"].isin(["Gametocytes (developing)","Gametocytes (branching)","Gametocytes (early male)","Gametocytes (late male)","Gametocytes (early female)","Gametocytes (late female)" ])].obs_names, "Early schizont": obj[obj.obs["stage_bfs"] == "early schizont"].obs_names})

## Early gam and asexual driver

In [ ]:
for g in vtk_g_bf1n2:
    g.compute_absorption_probabilities()

In [ ]:
for g in vtk_g_bf1n2:
    g.plot_absorption_probabilities(components ='2,3')

In [ ]:
vtk_g_bf1_dr=[]
vtk_g_bf1_dr.append(vtk_g_bf1n2[0])
vtk_g_bf1_dr.append(vtk_g_bf1n2[0])

Computing driver genes in the gametocyte(branching) cluster that are driving cells to the male cluster

In [ ]:
early_gam_drivers = []
for i, g in enumerate(vtk_g_bf1_dr):
    if i == 0:
        early_gam_drivers.append(g.compute_lineage_drivers(lineages="Gametocyte", return_drivers=True, cluster_key = 'stage_bfs', clusters= ['late ring','early trophozoite','Gametocytes (committed)']))
    else:
        early_gam_drivers.append(g.compute_lineage_drivers(lineages="Gametocyte", return_drivers=True))

In [ ]:
early_gam_drivers[1]

In [ ]:
for drvs in early_gam_drivers:
    print(drvs.index.tolist()[:10])

In [ ]:
for g in vtk_g_bf1_dr:
    g.plot_lineage_drivers("Gametocyte", n_genes=4, components='2,3')

Plotting top 4 driver genes per timepoint

Phase potraits highlighting behaviour of gene in 'Gametocytes (branching)' cluster. We should start seeing activation atleast in cells that will then on become female.

In [ ]:
for i, drvs in enumerate(early_gam_drivers):
    scv.pl.scatter(ef_scv_stochastic[0], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['late ring','early trophozoite','Gametocytes (committed)'])
    scv.pl.scatter(ef_scv_stochastic[0], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (developing)'])
    scv.pl.scatter(ef_scv_stochastic[0], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['late trophozoite'])

Phase potraits highlighting behaviour of gene in 'Gametocytes (early male)','Gametocytes (late male)' cluster. We should see activation

Summary table of the female driver genes in the 'Gametocytes (committed and developing)' for all timepoints

In [ ]:
early_gam_drivers_bf1_vkck=[]
for drvs in early_gam_drivers:
    early_gam_drivers_bf1_vkck.append(pd.merge(drvs, Pf_genes_cln, how='left', left_index=True, right_index=True))

In [ ]:
for drvs, cc in zip(early_gam_drivers_bf1_vkck, corr_clust):
    drvs.to_csv(processed_data + '../RoadtoUMAP/early_gam_drivers_bf1_vkck_' + cc +'.csv')

In [ ]:
for drvs in early_gam_drivers:
    print(drvs.index.tolist()[:10])

#### Asexual drivers

In [ ]:
asexual_drivers = []
for i, g in enumerate(vtk_g_bf1_dr):
    if i == 0:
        asexual_drivers.append(g.compute_lineage_drivers(lineages="Asexual", return_drivers=True, cluster_key = 'stage_bfs', clusters= ['late ring','early trophozoite','Gametocytes (committed)']))
    else:
        asexual_drivers.append(g.compute_lineage_drivers(lineages="Asexual", return_drivers=True))

In [ ]:
asexual_drivers[1]

In [ ]:
for drvs in asexual_drivers:
    print(drvs.index.tolist()[:10])

In [ ]:
for g in vtk_g_bf1_dr:
    g.plot_lineage_drivers("Asexual", n_genes=4, components='2,3')

In [ ]:
for i, drvs in enumerate(asexual_drivers):
    scv.pl.scatter(ef_scv_stochastic[0], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['late ring','early trophozoite','Gametocytes (committed)'])
    scv.pl.scatter(ef_scv_stochastic[0], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['late trophozoite'])
    scv.pl.scatter(ef_scv_stochastic[0], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (developing)'])

In [ ]:
asexual_drivers_bf1_vkck=[]
for drvs in asexual_drivers:
    asexual_drivers_bf1_vkck.append(pd.merge(drvs, Pf_genes_cln, how='left', left_index=True, right_index=True))

In [ ]:
for drvs, cc in zip(asexual_drivers_bf1_vkck, corr_clust):
    drvs.to_csv(processed_data + '../RoadtoUMAP/asexual_drivers_bf1_vkck_' + cc +'.csv')

In [ ]:
sc.pl.umap(ef_scv_stochastic[0], color='assignment2')

## Male female driver analysis

Assessing genes involved in fate determination towards the male and female clusters with higher resolution using chunks generated by pseudotime

Manually setting terminal states
Combining late males and early males as terminal state for day 5 and using late male for all the other timepoints
Using Early female as terminal state for day 5 and using late female for all the other days

In [ ]:
ef_scv_stochastic[0].obs.clusters_gradients

### Computing driver genes in the gametocyte(branching) cluster that are driving cells to the male and female cluster

#### To the male

In [ ]:
# male_drivers = vtk_g_bf1n2[1].compute_lineage_drivers(lineages="Male", return_drivers=True, cluster_key = 'stage_bfs', clusters= ['Gametocytes (developing)', 'Gametocytes (branching)'])

In [ ]:
vtk_g_bf2_dr=[]
vtk_g_bf2_dr.append(vtk_g_bf1n2[1])
vtk_g_bf2_dr.append(vtk_g_bf1n2[1])

In [ ]:
male_drivers = []
for i, g in enumerate(vtk_g_bf2_dr):
    if i == 0:
        male_drivers.append(g.compute_lineage_drivers(lineages="Male", return_drivers=True, cluster_key = 'stage_bfs', clusters= ['Gametocytes (developing)', 'Gametocytes (branching)']))
    else:
        male_drivers.append(g.compute_lineage_drivers(lineages="Male", return_drivers=True))

In [ ]:
male_drivers[1]

In [ ]:
for drvs in male_drivers:
    print(drvs.index.tolist()[:10])

Plotting top 4 driver genes per timepoint

In [ ]:
for g in vtk_g_bf2_dr:
    g.plot_lineage_drivers("Male", n_genes=4, components='2,3') 

Phase potraits highlighting behaviour of gene in 'Gametocytes (branching)' cluster. We should start seeing activation atleast in cells that will then on become female.

Phase potraits highlighting behaviour of gene in 'Gametocytes (early male)','Gametocytes (late male)' cluster. We should see activation

In [ ]:
for i, drvs in enumerate(male_drivers):
    scv.pl.scatter(ef_scv_stochastic[1], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (developing)','Gametocytes (branching)'])
    scv.pl.scatter(ef_scv_stochastic[1], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (early male)','Gametocytes (late male)'])
    scv.pl.scatter(ef_scv_stochastic[1], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (early female)','Gametocytes (late female)'])

Upset plots to assess driver genes in branching gams that is shared between day10 new and day 10 old datasets and the 2 datasets combined using retaining only genes positively correlated to absorption probabilities towards terminal males where P < 0.001 as qvalue is NA https://github.com/theislab/cellrank/issues/819

In [ ]:
for i, drvs in enumerate(male_drivers):
    scv.pl.umap(ef_scv_stochastic[1],  layer='Mu',  color=drvs.index[0:5].to_list(), components='2,3')

In [ ]:
male_drivers_bf2_vkck=[]
for drvs in male_drivers:
    male_drivers_bf2_vkck.append(pd.merge(drvs, Pf_genes_cln, how='left', left_index=True, right_index=True))

In [ ]:
for drvs, cc in zip(male_drivers_bf2_vkck, corr_clust):
    drvs.to_csv(processed_data + '../RoadtoUMAP/male_drivers_bf2_vkck_' + cc +'.csv')

#### To the female cluster

In [ ]:
female_drivers = []
for i, g in enumerate(vtk_g_bf2_dr):
    if i == 0:
        female_drivers.append(g.compute_lineage_drivers(lineages="Female", return_drivers=True, cluster_key = 'stage_bfs', clusters= ['Gametocytes (developing)', 'Gametocytes (branching)']))
    else:
        female_drivers.append(g.compute_lineage_drivers(lineages="Female", return_drivers=True))


In [ ]:
female_drivers[1]

In [ ]:
for drvs in female_drivers:
    print(drvs.index.tolist()[:10])

Plotting top 4 driver genes per timepoint

In [ ]:
for g in vtk_g_bf2_dr:
    g.plot_lineage_drivers("Female", n_genes=4, components='2,3') 

Phase potraits highlighting behaviour of gene in 'Gametocytes (branching)' cluster. We should start seeing activation atleast in cells that will then on become female.

Phase potraits highlighting behaviour of gene in 'Gametocytes (early male)','Gametocytes (late male)' cluster. We should see activation

In [ ]:
for i, drvs in enumerate(female_drivers):
    scv.pl.scatter(ef_scv_stochastic[1], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (developing)','Gametocytes (branching)'])
    scv.pl.scatter(ef_scv_stochastic[1], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (early female)','Gametocytes (late female)'])
    scv.pl.scatter(ef_scv_stochastic[1], drvs.index[0:4].to_list(), color=['stage_bfs'], add_outline=['Gametocytes (early male)','Gametocytes (late male)'])

Upset plots to assess driver genes in branching gams that is shared between day10 new and day 10 old datasets and the 2 datasets combined using retaining only genes positively correlated to absorption probabilities towards terminal males where P < 0.001 as qvalue is NA https://github.com/theislab/cellrank/issues/819

In [ ]:
for i, drvs in enumerate(female_drivers):
    scv.pl.umap(ef_scv_stochastic[1],  layer='Mu',  color=drvs.index[0:5].to_list(), components='2,3')

In [ ]:
female_drivers_bf2_vkck=[]
for drvs in female_drivers:
    female_drivers_bf2_vkck.append(pd.merge(drvs, Pf_genes_cln, how='left', left_index=True, right_index=True))

In [ ]:
for drvs, cc in zip(female_drivers_bf2_vkck, corr_clust):
    drvs.to_csv(processed_data + '../RoadtoUMAP/female_drivers_bf2_vkck_' + cc +'.csv')

Plotting top 4 driver genes per timepoint

In [ ]:
for drvs in female_drivers:
    scv.pl.umap(ef_scv_stochastic[1],  layer='velocity',  color=drvs.index[0:5].to_list(), components='2,3')

Upset plot for female

Upset plots to assess driver genes in branching gams that is shared between day10 new and day 10 old datasets and the 2 datasets combined using retaining only genes positively correlated to absorption probabilities towards terminal males where P < 0.001 as qvalue is NA. bug in scvelo https://github.com/theislab/cellrank/issues/819

In [ ]:
scv.pl.umap(ef_scv_stochastic[1], color='PF3D7_1438800', layer='Mu', legend_loc='right', components='2,3')

In [ ]:
scv.pl.umap(ef_scv_stochastic[0], color='PF3D7_1438800', layer='velocity', legend_loc='right', components='2,3')

In [ ]:
# scv.pl.umap(ef_scv[1], color='PF3D7_0918700', layer='unspliced', legend_loc='right', components='2,3')

In [ ]:
# scv.pl.umap(ef_scv[2], color='PF3D7_0424100', layer='velocity_u', legend_loc='right', components='2,3')

In [ ]:
scv.pl.umap(ef_scv_stochastic[0], color='PF3D7_0918700', layer='velocity', legend_loc='right', components='2,3')

In [ ]:
# ef_scv[2].obs.columns

In [ ]:
# for obj, dg in zip(ef_scv_stochastic, branching_gam_drivers_f):
#     scv.pl.umap(obj,  layer='unspliced',  color=dg.index[0:5].to_list(), components='2,3')

In [ ]:
scv.pl.scatter(ef_scv_stochastic[1], 'PF3D7_1438800', color='stage_bfs', add_outline=['Gametocytes (committed)', 'Gametocytes (developing)', 'Gametocytes (branching)'], legend_loc='right')

In [ ]:
scv.pl.scatter(ef_scv_stochastic[1], 'PF3D7_1438800', color='stage_bfs', legend_loc='right')

In [ ]:
scv.pl.scatter(obj, 'PF3D7_0603600', color='stage_bfs', add_outline=['Gametocytes (committed)', 'Gametocytes (developing)', 'Gametocytes (branching)'], legend_loc='right')

Fitting GAMs for assesment of gene expression clusters over latent time

In [ ]:
modelGAM_male_lineage = []
for obj in ef_scv_stochastic:
    model=cr.ul.models.GAM(obj)
    modelGAM_male_lineage.append(model)


In [ ]:
for i, obj in enumerate(ef_scv_stochastic):
    cr.pl.cluster_lineage(
            obj,
            model = modelGAM_male_lineage[i],
            genes=branching_gam_drivers_m[i].index[(branching_gam_drivers_m[i]['Late male_qval'] < 0.001) & (branching_gam_drivers_m[i]['Late male_corr'] > 0.2)].to_list(),
            lineage="Late male",
            time_key="pseudo_mg",
            show_progress_bar=False,
            recompute=True,
            sharey=True,
            covariate_key=['stage_bfs']
        )


In [ ]:
for i, obj in enumerate(ef_scv_stochastic):
    cr.pl.cluster_lineage(
            obj,
            model = modelGAM_male_lineage[i],
            genes=branching_gam_drivers_m[i].index[(branching_gam_drivers_m[i]['Late male_qval'] < 0.001) & (branching_gam_drivers_m[i]['Late male_corr'] < -0.2)].to_list(),
            lineage="Late male",
            time_key="pseudo_mg",
            show_progress_bar=False,
            recompute=True,
            sharey=True,
            covariate_key=['stage_bfs']
        )

In [ ]:
v3_days_all_fnl

In [ ]:
# plt.rcParams["figure.figsize"] = (120, 5)
for i, obj in enumerate(ef_scv_stochastic):
    cr.pl.gene_trends(
        obj,
        lineages=['Late female','Late male'],
        model=modelGAM_male_lineage[i],
        data_key="X",
        genes=branching_gam_drivers_m[i].index[(branching_gam_drivers_m[i]['Late male_qval'] < 0.0000000000001) & (branching_gam_drivers_m[i]['Late male_corr'] < -0.3)].to_list(),
        ncols=5,
        time_key="pseudo_mg",
        same_plot=True,
        hide_cells=True,
        figsize=(50, 6),
        n_test_points=200)
    
# plt.rcParams["figure.figsize"] = (7, 5)

In [ ]:
modelGAM_female_lineage_bf2 = []
for obj in ef_scv_stochastic:
    model=cr.ul.models.GAM(obj)
    modelGAM_female_lineage_bf2.append(model)


In [ ]:
modelGAM_female_lineage_bf2 = []
for obj in ef_scv_stochastic:
    model=cr.ul.models.GAM(obj)
    modelGAM_female_lineage_bf2.append(model)

Driver genes in CD positively correlated with progression to female fate

Driver genes in gambranch positively correlated with progression to female fate

In [ ]:
for i, obj in enumerate(ef_scv_stochastic):
    cr.pl.cluster_lineage(
            obj,
            model = modelGAM_female_lineage_bf2[i],
            genes=branching_gam_drivers_f[i].index[(branching_gam_drivers_f[i]['Late female_qval'] < 0.0000000000001) & (branching_gam_drivers_f[i]['Late female_corr'] > -0.2)].to_list(),
            lineage="Late female",
            time_key="pseudo_mg",
            show_progress_bar=False,
            recompute=True,
            sharey=True,
            covariate_key=['stage_bfs']
        )

branching_gam_drivers_f is the equivalent of female_drivers_CDB for the branching driver list. Care should be taken to avoid being confused by names

In [ ]:
##Pseudotime kernel